<a href="https://colab.research.google.com/github/sindla97/Machine-Learning/blob/main/humana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
import numpy as np

In [79]:
df=pd.read_csv("2022_Competition_Training.csv", low_memory=False)

In [80]:
df.head()

,id,rx_hum_19_pmpm_cost,cmsd2_can_mal_male_genital_pmpm_ct,cmsd2_inj_surgical_pmpm_ct,rev_pm_capd_pmpm_cd_ct,cci_chf_pmpm_ct,rx_phar_cat_safeway_pmpm_ct,cmsd2_ext_car_pmpm_ct,cmsd2_dig_liver_pmpm_ct,cmsd2_mus_arthropath_pmpm_ct,...,cmsd2_ext_nature_pmpm_ct,cmsd2_gus_m_genital_pmpm_ct,atlas_totalpopacs,cmsd2_neo_neo_other_pmpm_ct,cmsd2_dig_hernia_pmpm_ct,cmsd2_sns_blood_pmpm_ct,cmsd2_ext_compl_medical_care_pmpm_ct,rwjf_food_env_inx,atlas_snapspth16,hi_flag
0,100039218.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.08,33511.0,0.0,0.0,0.00,0.0,9.1,NaN,0
1,100042427.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,176797.0,0.0,0.0,0.00,0.0,6.8,0.670459,0
2,100055103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,380673.0,0.0,0.0,0.00,0.0,8.3,0.879186,0
3,100074556.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,267696.0,0.0,0.0,0.00,0.0,7.3,0.746201,0
4,100079932.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,19095.0,0.0,0.0,0.25,0.0,6.9,NaN,0


In [81]:
df.shape

(48300, 881)

In [82]:
df_null=pd.DataFrame(df.isnull().sum()).iloc[[x for x in range(df.shape[1]) if df.isnull().sum()[x] > 0.50*(df.shape[0])],:]


<ipython-input-82-b90e7bb9022c>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_null=pd.DataFrame(df.isnull().sum()).iloc[[x for x in range(df.shape[1]) if df.isnull().sum()[x] > 0.50*(df.shape[0])],:]


In [83]:
df_null = df_null.reset_index()

In [84]:
df['hi_flag'].value_counts()

,count
hi_flag,
0,46182
1,2118


In [85]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['cons_mobplus'] = le.fit_transform(df['cons_mobplus'])
df['cms_ra_factor_type_cd'] = le.fit_transform(df['cms_ra_factor_type_cd'])
df['cons_homstat'] = le.fit_transform(df['cons_homstat'])
df['sex_cd'] = le.fit_transform(df['sex_cd'])
df['lang_spoken_cd'] = le.fit_transform(df['lang_spoken_cd'])
df['rucc_category'] = le.fit_transform(df['rucc_category'])
df['cms_race_cd'] = le.fit_transform(df['cms_race_cd'])



In [86]:
 df[['cons_mobplus','cms_ra_factor_type_cd', 'cons_homstat', 'sex_cd', 'lang_spoken_cd','rucc_category','cms_race_cd']].head()


,cons_mobplus,cms_ra_factor_type_cd,cons_homstat,sex_cd,lang_spoken_cd,rucc_category,cms_race_cd
0,0,3,1,1,1,2,2
1,0,3,4,0,1,0,7
2,2,3,3,0,3,1,2
3,4,2,5,0,3,1,3
4,4,4,5,1,3,4,2


In [87]:
columns = [x for x in df.columns if x not in ['id','hi_flag']+list(df_null.index)]
len(columns)

879

In [88]:
from sklearn.model_selection import *
X_train, X_test, y_train, y_test = train_test_split(
    df[columns], df['hi_flag'], test_size=0.2, random_state=42, stratify=df['hi_flag'])


In [89]:
y_test.sum()/(y_train.sum() + y_test.sum())

0.20018885741265344

In [90]:
len(y_train), y_train.sum()

(38640, 1694)

In [91]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [92]:
param_grid = {'n_estimators':[100,250, 500,750, 1000,1250,1500],
              'max_depth':[4, 5, 6, 7,8, 9],
              'learning_rate':[0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.00001],'reg_lambda':[1, 10, 100,500, 1000, 2000], 'reg_alpha':[0, 0.25,0.5,0.75, 1] ,
              'colsample_bytree':[0.5,0.6, 0.7,0.8,0.9],'subsample':[0.5, 0.6, 0.7,0.8,0.9],
              'min_samples_leaf':[5,50,100,200,250]}
sample_weight=np.where(df['hi_flag'] == 1, df['hi_flag'].shape[0]/df['hi_flag'].sum(), 1)

In [97]:
bst=XGBClassifier(objective='binary:logistic',eval_metric='auc', verbosity=2)
bst_cv=RandomizedSearchCV(bst,param_grid, n_iter=3)
bst_cv.fit(df[columns], df['hi_flag'],sample_weight )



RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='auc',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=Non...
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=3,
                   param_distributions={'colsample_bytree': [0.5, 0.6, 0.7, 0.8,
                                                             0.9],
                                        'learning_rate': [0.1, 0.05, 0.01,
                                                          0.005, 0.001, 0.0005,
                                                          1e-05],
                                        'max_depth': [4, 5, 6, 7, 8, 9],
                                        'n_estimators': [100, 250, 500, 750,
                                                         1000, 1250, 1500],
                                        'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
                                        'reg_lambda': [1, 10, 100, 500, 1000,
                                                       2000],
                                        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9]})

In [109]:
pd.DataFrame(bst_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_reg_lambda,param_reg_alpha,param_n_estimators,param_max_depth,param_learning_rate,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,52.727408,1.220714,0.446868,0.086453,0.7,2000,0,500,4,0.0005,0.6,"{'subsample': 0.7, 'reg_lambda': 2000, 'reg_al...",0.956211,0.956211,0.956108,0.956108,0.956108,0.956149,0.000051,1
1,50.001384,1.870879,0.454836,0.073075,0.5,500,0.25,250,9,0.001,0.9,"{'subsample': 0.5, 'reg_lambda': 500, 'reg_alp...",0.956211,0.956211,0.956108,0.956108,0.956108,0.956149,0.000051,1
2,51.741998,1.479644,0.501981,0.105694,0.6,500,0,250,9,0.005,0.5,"{'subsample': 0.6, 'reg_lambda': 500, 'reg_alp...",0.956211,0.956211,0.956108,0.956108,0.956108,0.956149,0.000051,1


In [102]:
bst_cv.best_params_

{'subsample': 0.7,
 'reg_lambda': 2000,
 'reg_alpha': 0,
 'n_estimators': 500,
 'max_depth': 4,
 'learning_rate': 0.0005,
 'colsample_bytree': 0.6}

In [103]:
#Single_model
bst = XGBClassifier(n_estimators=500, max_depth=4, learning_rate=0.0005,reg_lambda= 2000,objective='binary:logistic', reg_alpha=0,
                    colsample_bytree=0.6,subsample=0.7,eval_metric='auc', verbosity=1)
# fit model
sample_weight=np.where(y_train == 1, len(y_train)/y_train.sum(), 1)
# sample_weight.max(), sample_weight.mean(), sample_weight.sum()
bst.fit(X_train, y_train, sample_weight)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:723: FutureWarning: Pass `sample_weight` as keyword args.
  warnings.warn(msg, FutureWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0005, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [104]:
# make predictions
preds = bst_cv.predict(X_test)
preds_train = bst_cv.predict(X_train)


In [28]:
preds_scroe = bst.score(X_test,y_test)
preds_train_scroe = bst.score(X_train, y_train)

In [105]:
from sklearn.metrics import *
accuracy = accuracy_score(y_test, preds)
accuracy_train = accuracy_score(y_train, preds_train)
print(f'test_acc : {accuracy * 100.0}, train_acc:{accuracy_train*100.0}')

test_acc : 95.61076604554866, train_acc:95.61594202898551


In [106]:
confusion_matrix(y_test, preds), confusion_matrix(y_train, preds_train)

(array([[9236,    0],
        [ 424,    0]]),
 array([[36946,     0],
        [ 1694,     0]]))

In [62]:
auc = roc_auc_score(y_test, preds)
auc_train = roc_auc_score(y_train, preds_train)
print(f'test_auc : {auc * 100.0}, train_auc:{auc_train*100.0}')

test_auc : 65.78518635037629, train_auc:79.3658200286055


In [31]:
print(classification_report(y_train, preds_train))

              precision    recall  f1-score   support

           0       0.99      0.74      0.85     36946
           1       0.13      0.84      0.23      1694

    accuracy                           0.75     38640
   macro avg       0.56      0.79      0.54     38640
weighted avg       0.95      0.75      0.82     38640



In [66]:
df_col = pd.DataFrame(bst.feature_importances_, columns).reset_index()
df_col.sort_values(0, ascending=False,inplace=True)

In [67]:
df_col.head(10)

,index,0
402,cons_stlindex,0.026159
399,cms_orig_reas_entitle_cd,0.020131
101,cons_hxmioc,0.013947
535,cms_low_income_ind,0.009267
70,cms_disabled_ind,0.008340
695,total_physician_office_allowed_pmpm_cost,0.006241
378,cons_homstat,0.006083
733,cms_frailty_ind,0.005373
287,cons_stlnindx,0.004994
393,cms_dual_eligible_ind,0.004641
